In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA, KernelPCA
from sklearn.ensemble import RandomForestClassifier
import optuna
import optuna.visualization as vis

n = 9

X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

def objective(trial):
    ### ========== Shuffle and Split Data ========== ###

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.8, random_state=42)  # 60% training
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.8, random_state=42)  # 20% validation, 20% test

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Suggest dimensionality reduction and kernel parameters
    use_pca = trial.suggest_categorical('use_pca', ['True', 'False'])
    n_components_pca = trial.suggest_int('n_pca_components', 1, max(1, X_train_scaled.shape[1]))
    kernel_pca = trial.suggest_categorical('pca_kernel', ['linear', 'poly', 'rbf', 'sigmoid', 'cosine'])

    ### ========== Dimensionality Reduction ========== ###

    if use_pca == 'True':
        pca = KernelPCA(
            kernel=kernel_pca,
            n_components=n_components_pca,
            n_jobs=-1
        )
        X_train_scaled = pca.fit_transform(X_train_scaled)
        X_val_scaled = pca.transform(X_val)
        X_test_scaled = pca.transform(X_test)

    # KNN Classifier parameters
    classifier = RandomForestClassifier(
        n_estimators=trial.suggest_int('n_estimators', 100, 300, step=25), 
        max_depth=trial.suggest_int('max_depth', 20, 50),
        max_features=trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        criterion=trial.suggest_categorical('criterion', ['entropy', 'log_loss']),
        n_jobs=-1
    )

    # Train the classifier
    classifier.fit(X_train_scaled, y_train)

    # Evaluate on the validation set
    y_val_pred = classifier.predict(X_val_scaled)
    val_accuracy = accuracy_score(y_val, y_val_pred)

    # Report validation accuracy for pruning
    trial.report(val_accuracy, step=0)

    # Check if the trial should be pruned
    if trial.should_prune():
        raise optuna.TrialPruned()

    # Evaluate on the test set
    y_test_pred = classifier.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    print(f"Test Accuracy: {test_accuracy:.4f}")

    return val_accuracy

# Create a study object with a pruner
pruner = optuna.pruners.MedianPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)

# Optimize the objective function
study.optimize(objective, n_trials=100)

print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

# Visualize the optimization history and parameter importances
vis.plot_optimization_history(study).show()
vis.plot_param_importances(study).show()
vis.plot_slice(study).show()